# Models visualised

In [ ]:
import datetime as dt
import numpy as np
import xarray as xr
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# from scipy.interpolate import interp1d
# import cartopy.crs as ccrs
# import cartopy.feature as cfeature
import holoviews as hv
import hvplot.xarray
import panel as pn

import eoxmagmod
from chaosmagpy.plot_utils import nio_colormap

In [ ]:
hv.extension("bokeh")

In [ ]:
def grid(nlats=179, nlons=360, height=0):
    """A global grid of the specified size"""
    _lats = np.linspace(-89, 89, nlats)
    _lons = np.linspace(-179, 180, nlons)
    coords = np.empty((_lats.size, _lons.size, 3))
    coords[:,:,1], coords[:,:,0] = np.meshgrid(_lons, _lats)
    coords[:,:,2] = height # height above WGS84 in km
    return coords

def eval_model(
        time=dt.datetime(2020, 1, 1),
        coords=grid(),
        shc_model=eoxmagmod.data.IGRF13,
    ):
    """Evaluate a .shc model at a fixed time

    Args:
        time (datetime)
        coords (ndarray)
        shc_model (str): path to file

    Returns:
        dict: magnetic field vector components:
            https://intermagnet.github.io/faq/10.geomagnetic-comp.html
    """
    # Convert Python datetime to MJD2000
    epoch = eoxmagmod.util.datetime_to_decimal_year(time)
    mjd2000 = eoxmagmod.decimal_year_to_mjd2000(epoch)
    # Load model
    model = eoxmagmod.load_model_shc(shc_model)
    # Evaluate in North, East, Up coordinates
    # Specify coordinate systems according to:
    #   CT_GEODETIC_ABOVE_WGS84 = 0,
    #   CT_GEOCENTRIC_SPHERICAL = 1,
    #   CT_GEOCENTRIC_CARTESIAN = 2
    # https://github.com/ESA-VirES/MagneticModel/blob/staging/eoxmagmod/eoxmagmod/pymm_coord.h
    input_coordinate_system = 0
    output_coordinate_system = 0
    b_neu = model.eval(mjd2000, coords, input_coordinate_system, output_coordinate_system)
    # Inclination (I), declination (D), intensity (F)
    inc, dec, F = eoxmagmod.vincdecnorm(b_neu)
    return {"X": b_neu[:,:,0], "Y": b_neu[:,:,1], "Z": -b_neu[:,:,2],
            "I": inc, "D":dec, "F":F}

def eval_model_on_grid(model=eoxmagmod.data.IGRF13, heights=range(0, 1000, 100), nlats=180, nlons=360):
    ds_set = []
    for height in heights:
        coords=grid(nlats, nlons, height)
        output = eval_model(coords=coords, shc_model=model)
        _ds = xr.Dataset(
            {
                "Z": (("y", "x"), output["Z"]),
                "X": (("y", "x"), output["X"]),
                "Y": (("y", "x"), output["Y"]),
            },
            coords={"lat": (("y", "x"), coords[:,:,0]), "lon": (("y", "x"), coords[:,:,1])}
        )
        _ds = _ds.assign_coords({"height": height})
        ds_set += [_ds]
    ds = xr.concat(ds_set, dim="height")
    ds["height"].attrs = {"units": "km"}
    return ds

## Core field

In [ ]:
@pn.cache
def get_core_field():
    return eval_model_on_grid(
        model=eoxmagmod.data.IGRF13,
        heights=range(-3000, 3000, 500)
    )

ds_igrf = get_core_field()
# ds_igrf

In [ ]:
# hv_core_field = ds_igrf.hvplot.contourf(x="lon", y="lat", z="Z", levels=30, clim=(-60_000, 60_000), xlim=(-180, 180), ylim=(-90, 90))
hv_core_field = ds_igrf.hvplot.contourf(
    x="lon", y="lat", z=["X", "Y", "Z"],
    levels=50, xlim=(-180, 180), ylim=(-90, 90), cmap="bwr"
)
# hv_core_field

In [ ]:
dash_core = pn.panel(hv_core_field)
dash_core[1][0].value = "Z"
dash_core[1][1].value = 0
# dash_core

## Lithospheric field

In [ ]:
@pn.cache
def get_crust_field():
    return eval_model_on_grid(
        model=eoxmagmod.data.CHAOS_STATIC_LATEST,
        heights=range(0, 500, 100),
        nlats=180,
        nlons=360,
    )

ds_crust = get_crust_field()
# ds_crust

In [ ]:
hv_crust_field = ds_crust.hvplot.contourf(
    x="lon", y="lat", z=["X", "Y", "Z"],
    levels=100, xlim=(-180, 180), ylim=(-90, 90), cmap="bwr", clim=(-150, 150),
)
# hv_crust_field

In [ ]:
dash_crust = pn.panel(hv_crust_field)
dash_crust[1][0].value = "Z"
dash_crust[1][1].value = 0
# dash_crust

## Compile dashboard

In [ ]:
# spinner = pn.indicators.LoadingSpinner()
# pn.state.add_periodic_callback(lambda: spinner.param.set_param(value=pn.state.param.busy), period=100)

vires_core_url = "https://vires.services/?ws=data%3Aapplication/json%2Bgzip%3Bbase64%2CH4sIAMta5mMC/51Va0/iQBT9K6RfF5p5dF5%2BQ5R1E18Rd1UMaSodcTalJW1hNab/fe%2BMPEpZXSWZQOfMPefc3t7evnqFzhdmrH/pvDBZ6h20POpj7COv3fJKM9UDnehx%2BXZ07xGEeQdhWNcIHbjlI4SGNrp2RugBU7B8pdTQGy2ljrJpZNY6rINJh0IseU%2BHXKOtM6uzMPpPPSVvkmQP2tJmeRbPx2XRy9JHM5nn0TLi1RvchBeXx1fhWfd7Nzy9CvGhgxemMA%2BJhuvHKCk0SGTzMjGpLurQLBqb8gUQbD2iPJrqEmrlFA7D8%2BOeu8qjdGKV7jGDVNstZv9svuMsyfJiHDkf77cuvQrQ/ldJABburnUMUJnPtZVx/mGui24jiw63evyjFNbc8CyLdbLNB7Jg75ErS9%2BQNmW0aW2VDPlit2j9fQp2spMf/Wqxbhu%2ByPkS%2BqHvXaOuwpHEPzmxWeh8YtJJiB132OAS5/cZ5o8GUwJRfoJ3tCevb3QSrzq/XqN2K0Af1uc2PL5pmDJ4mP%2B3vNuXONyLWLnj6SxLdVrau7x/9YztDu%2BsdxEOTrohOXJ9YyZuqnzzdhq3qkZvMoDl0WVWmPWAmW020M8MSYqVLyQnmFLebnUIx0hwRnwWMBoQJaF9A0wQ44GPhSScBNzeQGzyzaztIJ9SFRCOFGJSAhNoyGcCCcE5SAbwS%2BARQaAIAGJcEgSWEG7F5rOlClIgISRMXImoVJJYGcEVZxQLLlQgqGAW4xRjKQVWQiglFLUquZk8le6V8xVlAmNEGZFMKY4thUibHkgoCg5KCpcOQRyMFKyAQP6SjWzhnrvPptia3acwo8t5/Da%2Bk6xsjm54SC%2BW5D4ZfTf/3f40enDDJ50nicXIKmi03jVCXGusoiw2qoPEcSHHLb/acBztuHxK0GJFVOokMeXy3eoms6eo9n05zKNFVtv3nqI8MfWP0vmgV9v9hDpF8Qqoqr%2BP2UcCwQcAAA%3D%3D"
vires_crust_url = "https://vires.services/?ws=data%3Aapplication/json%2Bgzip%3Bbase64%2CH4sIAMta5mMC/51Va0/iQBT9K6RfF5p5dF5%2BQ5TVBF0j7qoY0lQ64mxKS9rCagj/fe%2BMPEpZXSWZQOfMPefcuZ3eWXiFzudmpH/pvDBZ6h01POpj7COv2fBKM9F9nehR%2Bbb04BGEeQthGDcIHbnhI4QGNrqyRugRUzB8pdTAG66kTrJJZDY6rIVJi0IseU%2BH3KCdNaszN/pPNSVvnGSP2tKmeRbPRmXRydInM57l0Spi4fVvwx9Xp9fhRft7O%2Bxdh/jYwXNTmMdEw/NTlBQaJLJZmZhUF1VoGo1M%2BQoIth5RHk10CbVyCsfh5WnHPeVROrZKD5hBqs0Gs38231GWZHkxipyP91uX3hLQ7ldJABZu1zoGqMxn2so4/zDXRbuWRYtbPf5RChtueJHFOtnlA1mw98hLS9%2BStmW0ae2UDPliv2jdQwp2tpcfxl%2Bp1F3NFDlTQj80va8VVTiS%2BCcnNnOdj006DrHjDmpc4vw%2BwzyvMSUQ5Sd4JwfyukYn8frYV2vUbATow/rchae3NVMGb/L/lveHEgcHEZdueTLNUp2WdpcPC8/Y0%2BFd9M7D/lk7JCfu3JixaynfvL1Tu1wO32QAy6OrrDCb7jLdTuAwMyQpVr6QnGBKebPRIhwjwRnxWcBoQJRsNmiACWI88LGQhJOA2w3EJt822hbyKVUB4UghJiUwgYZ8JpAQnINkAL8EXhEEigAgxiVBYAnhVmw2XakgBRJCQruViEoliZURXHFGseBCBYIKZjFOMZZSYCWEUkJRq5Kb8XPpvjdfUSYwRpQRyZTi2FKItOmBhKLgoKRw6RDEwUjBCAjkL9nQFu6l/WKKncbdgwZdzuK33p1kZb1vw0t6tSR3X3Rd83fzXvToOk86SxKLkXXQcDOrhbijsY6y2LAKEseFHHf8Kp1xuOfyKUGLFVGpk8SUq2%2BrnUyfo8rlcpxH86wy7zxHeWKqN9Jlv1OZ/YQ6RfEaWC7/AkvRHGW%2BBwAA"
dash = pn.Column(
    # spinner,
    pn.pane.Markdown(f"## Core field:\n[(View in VirES)]({vires_core_url})"),
    dash_core,
    pn.pane.Markdown(f"## Crustal field:\n[(View in VirES)]({vires_crust_url})"),
    dash_crust,
)

In [ ]:
# bug when running? run this twice to fix it
dash.servable(title="Geomagnetic Model Explorer")